In [ ]:
# Langkah 1 : Load Data Latih dan data uji

import pandas as pd
import joblib

# Load data
def load_data(latih_path, uji_path):
    data_latih = pd.read_excel(latih_path)
    data_uji = pd.read_excel(uji_path)
    return data_latih, data_uji

latih_path = 'data_latih.xlsx'
uji_path = 'data_uji.xlsx'

data_latih, data_uji = load_data(latih_path, uji_path)
data_latih.head(), data_uji.head()


In [ ]:
# Langkah 2 :  Persiapan Training Data dari Data Excel
def prepare_training_data(data_latih):
    X_train = data_latih.drop(columns=['Skenario', 'Label', 'Banjir Historis'])
    y_train = data_latih['Label']
    feature_names = X_train.columns.tolist()
    return X_train, y_train, feature_names

X_train, y_train, feature_names = prepare_training_data(data_latih)
X_train.head(), y_train.head()


In [ ]:
# Langkah 3 :  Proses Training Data untuk pembentukan model random forest
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model
def train_random_forest(X_train, y_train):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    return rf_model

rf_model = train_random_forest(X_train, y_train)
print("Training Siap !")


In [ ]:
# Langkah 4 : Simpan Model Random Forest
def save_model(model, file_name):
    joblib.dump(model, file_name)

save_model(rf_model, 'rf_model.pkl')
print("Model Disimpan !")


In [ ]:
# Langkah 5 : Simpan Rules Random Forest

import matplotlib.pyplot as plt
from sklearn.tree import plot_tree, _tree
import os

# Save the decision tree as a PNG file
def save_decision_tree_as_png(tree, feature_names, class_names, file_name):
    plt.figure(figsize=(20,10))
    plot_tree(
        tree,
        feature_names=feature_names,
        class_names=class_names,
        filled=True,
        rounded=True,
        proportion=True,
        fontsize=10
    )
    plt.savefig(file_name)
    plt.close()

# Extract rules from a tree
def extract_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    rules = []

    def recurse(node, depth, path):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            path_left = f"{path} AND {name} <= {threshold:.2f}"
            path_right = f"{path} AND {name} > {threshold:.2f}"
            recurse(tree_.children_left[node], depth + 1, path_left)
            recurse(tree_.children_right[node], depth + 1, path_right)
        else:
            value = tree_.value[node]
            class_name = class_names[value.argmax()]
            rule = f"IF {path} THEN {class_name}"
            rules.append(rule)

    recurse(0, 1, "True")
    return rules

# Save rules to a text file
def save_rules(rules, file_name):
    with open(file_name, 'w') as f:
        for rule in rules:
            f.write(f"{rule}\n")

class_names = rf_model.classes_.tolist()

os.makedirs('decision_trees', exist_ok=True)
for i in range(min(len(rf_model.estimators_), 10)):  # Save up to the first 10 trees
    file_name = f'decision_trees/tree_{i + 1}.png'
    save_decision_tree_as_png(rf_model.estimators_[i], feature_names, class_names, file_name)
    print(f'Saved {file_name}')
    
    rules = extract_rules(rf_model.estimators_[i], feature_names, class_names)
    save_rules(rules, f'decision_trees/rules_{i + 1}.txt')
    print(f'Saved rules for tree {i + 1}')


In [ ]:
# Langkah 6 : Evaluasi Model dan tes akurasi dari model

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Evaluate the model
def evaluate_model(model, X_train, y_train):
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_pred)
    train_report = classification_report(y_train, train_pred)
    conf_matrix = confusion_matrix(y_train, train_pred)
    return cv_scores, train_accuracy, train_report, conf_matrix

cv_scores, train_accuracy, train_report, conf_matrix = evaluate_model(rf_model, X_train, y_train)

# Display evaluation results
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean Cross-Validation Score: {cv_scores.mean()}")
print(f"Training Accuracy: {train_accuracy}")
print(f"Classification Report:\n{train_report}")
print(f"Confusion Matrix:\n{conf_matrix}")


In [ ]:
# Langkah 7 : Melakukan persiapan proses Klasifikasi Data Uji
def prepare_testing_data(data_uji):
    X_test = data_uji.drop(columns=['Kecamatan', 'Latitude', 'Longitude', 'Banjir Historis'])
    kecamatan = data_uji['Kecamatan']
    latitude = data_uji['Latitude']
    longitude = data_uji['Longitude']
    return X_test, kecamatan, latitude, longitude

X_test, kecamatan, latitude, longitude = prepare_testing_data(data_uji)
X_test.head()


In [ ]:
# Langkah 8 : Proses Prediksi
def make_predictions(model, X_test):
    return model.predict(X_test)

y_pred = make_predictions(rf_model, X_test)

# Save prediction results to an Excel file and return results DataFrame
def save_prediction_results(kecamatan, y_pred, file_name):
    results = pd.DataFrame({
        'Kecamatan': kecamatan,
        'Prediksi': y_pred
    })
    results.to_excel(file_name, index=False)
    return results

results = save_prediction_results(kecamatan, y_pred, 'hasil_prediksi.xlsx')
results


In [ ]:
# Langkah 9 : Proses Visualisasi Data Menggunakan Maps

import folium
from folium.plugins import MarkerCluster
from IPython.display import display

# Create a map with Folium
def create_folium_map(kecamatan, latitude, longitude, predictions=None, file_name=None):
    # Create a map centered around the average location
    m = folium.Map(location=[latitude.mean(), longitude.mean()], zoom_start=10)
    marker_cluster = MarkerCluster().add_to(m)

    # Color dictionary for different severity levels
    color_dict = {
        'Aman': 'green',
        'Waspada': 'orange',
        'Siaga': 'red',
        'Awas': 'darkred'
    }

    # Add markers to the map
    for i in range(len(kecamatan)):
        if predictions is not None:
            color = color_dict[predictions[i]]
            popup_text = f"{kecamatan[i]}: {predictions[i]}"
        else:
            color = 'blue'
            popup_text = kecamatan[i]
        
        folium.Marker(
            location=[latitude[i], longitude[i]],
            popup=popup_text,
            icon=folium.Icon(color=color)
        ).add_to(marker_cluster)

    # Save the map to an HTML file
    if file_name:
        m.save(file_name)

    return m

# Display map before predictions
map_before = create_folium_map(kecamatan, latitude, longitude)
print("Peta Sebelum Klasifikasi")
display(map_before)

# Create and display map after predictions
map_after = create_folium_map(kecamatan, latitude, longitude, y_pred, 'peta_prediksi.html')
print("Peta Sesuadah Klasifikasi")
display(map_after)
print("Peta prediksi telah disimpan dalam file 'peta_prediksi.html'")
